In [ ]:
import sys; sys.path.append('..')
import demosaicing as dm
import importlib; importlib.reload(dm)

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

## Validation over Kodak dataset

In [ ]:
demosaic_algos = [
    ('opencv_bilinear',     lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR)),
    ('opencv_vng',          lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_VNG)),
    ('opencv_edge_aware',   lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_EA)),
    ('bilinear',            dm.demosaic_bilinear),
    ('smooth_hue',          dm.demosaic_smooth_hue),
    ('median_filter',       dm.demosaic_median_filter),   
    ('laplacian_corrected', dm.demosaic_laplacian_corrected)
]

history = {}
print(f"{'method':>20}\t{'mean_pnsr':>20}\t{'median_psnr':>20}")
for algo, f in demosaic_algos:
    history[algo] = dm.validate_kodak(f)
    print(f"{algo:>20}\t{np.mean(history[algo]['psnr']):>20.3f}\t{np.median(history[algo]['psnr']):>20.3f}")

In [ ]:
img_number = [18,19,22]
print(f"{'method':>20}"+"".join([f"{i:>20}" for i in img_number]))
for algo,hist in history.items():
    d = ""
    for i in img_number:
        d = d + f"{np.mean(hist['psnr'][i]):>20.3f}"
    print(f"{algo:>20}\t"+d)